In [ ]:
import nltk
import json
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
nltk.download('punkt')

with open('/content/wocka.json','r') as f:
    temp=json.load(f)

In [ ]:
jokes=list(map(lambda x: x['body'],temp))
jokes=list(map(lambda x: x.lower(),jokes)) #lowercase

sentences=list()
for i in jokes:
x=sent_tokenize(i)
sentences.append(x)

words=list()
for i in jokes:
    x=word_tokenize(i);
    words.append(x)

In [ ]:
nltk.download("stopwords")
from nltk.corpus import stopwords

from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer

nltk.download('wordnet')
nltk.download('omw-1.4')

In [ ]:
stoplist=stopwords.words('english')
stoplist=set(stoplist)
words_filtered=list()
for i in words:
    x=[s for s in i if s not in stoplist]
    words_filtered.append(x)

words_stem=list()
ps=PorterStemmer()
for i in words_filtered:
    x=[]
    for j in i:
        y=ps.stem(j)
        x.append(y)
    x=[w for w in x if w.isalnum()]
    words_stem.append(x)

words_lem=list()
wl=WordNetLemmatizer()
for i in words_filtered:
    x=[]
    for j in i:
        j1=wl.lemmatize(j,pos='n')
        j2=wl.lemmatize(j1,pos='v')
        j3=wl.lemmatize(j2,pos='a')
        j4=wl.lemmatize(j3,pos='r')
        x.append(j4)
    y=[w for w in x if w.isalnum()]
    words_lem.append(y)

In [ ]:
def create_inv_index(corpus: list):
    inv_index={}
    for docid,text in enumerate(corpus):
        for i in text:
            if i not in inv_index.keys():
                inv_index[i]=[docid]
            elif docid not in inv_index[i]:
                inv_index[i].append(docid)
    return inv_index

inv_index=create_inv_index(words_lem)
sorted_terms=sorted(inv_index.keys())
final_inv_index={i : inv_index[i] for i in sorted_terms}

In [ ]:
def bool_and(l1,l2):
    result=[]
    for i in l1:
        if i in l2:
            result.append(i)
    return result

def bool_or(l1,l2):
    result=[]
    for i in l1:
        result.append(i)
    for i in l2:
        if(i not in result):
            result.append(i)
    return result

def bool_not(l1):
    result=[]
    for i in range(10019):
        if(i not in l1):
            result.append(i)
    return result

l1=final_inv_index["cat"]
l2=final_inv_index["car"]
print("boolean and :" , bool_and(l1,l2))
print("boolean or :" , bool_or(l1,l2))
print("boolean not :" , bool_not(l1))


In [ ]:
pos_index={}
def create_pos_index(corpus: list):
    docno, lineno= 0, -1
    for doc in corpus:
        lineno += 1
        for pos,term in enumerate(doc):
            if term in pos_index:
                pos_index[term][0]=pos_index[term][0]+1
                if docno in pos_index[term][1]:
                    pos_index[term][1][lineno].append(pos)
                else:
                    pos_index[term][1][lineno]=[pos]
            else:
                pos_index[term]=[]
                pos_index[term].append(1)
                pos_index[term].append({})
                pos_index[term][1][lineno]=[pos]
        docno += 1
    return pos_index

create_pos_index(words_lem)
sorted_terms=sorted(pos_index.keys())
final_pos_index={i : pos_index[i] for i in sorted_terms}


In [ ]:
def merge(l1,l2):
result=[]
    for d1 in l1[1].keys():
        if d1 in l2[1].keys():
            for i in l1[1][d1]:
                for j in l2[1][d1]:
                    if(i<j):
                        result.append(d1)
    return result

def phrase_query():
    print("enter a phrase query")
    phrase=input()
    phrase=list(phrase.split());
    result=[]
    for i in range(len(phrase)-1):
        l1=final_pos_index[phrase[i]]
        l2=final_pos_index[phrase[i+1]]
        temp=merge(l1,l2)
        if len(result):
            result=bool_and(result,temp)
        else:
            result = temp
    return result

In [ ]:
def create_permuterm_index(corpus):
    permuterm_index={}
    for doc in corpus:
        for term in doc:
            term += "$"
            for i in range(len(term)):
                permuterm=str(term[i:]+term[:i])
                if permuterm not in permuterm_index.keys():
                    permuterm_index[permuterm]=set()
                permuterm_index[permuterm].add(str(term[:-1]))
    return permuterm_index

permuterm_index=create_permuterm_index(words_lem)

In [ ]:
def wildcard_query():
    print("enter a wildcard query")
    wcquery=input()
    wcquery+="$"
    if(wcquery[len(wcquery)-1]!='*'):
        for i in range(len(wcquery)):
            wcquery = str(wcquery[1:]+wcquery[:1])
            if wcquery[len(wcquery)-1] == '*':
                break
    for i in permuterm_index.keys():
        if i.startswith(str(wcquery[:-1])):
            print(list(permuterm_index.get(i))[0],final_inv_index[(list(permuterm_index.get(i))[0])])